<a href="https://colab.research.google.com/github/Ethrogo/dssHW/blob/main/chatHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 0: Ensure TensorFlow 2.x is installed
# !pip install tensorflow

# Step 1: Import necessary libraries

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np


In [7]:
# Example dataset: A small set of question-answer pairs
questions = ['Who is your favorite athlete?', 'Who is the best NBA team?', 'Hi', 'Hello', 'hi', 'hello', 'How are you?', 'What is your name?', 'What is TensorFlow?', 'Do you like Python?', 'Goodbye', 'Im done', 'I want to quit' ]
answers = ['Zydrunas Ilgauskas', 'The New York Knicks', 'How can I help?', 'Hello', 'Hi', 'Hi, how can I help?', 'I am fine.', 'I am a bot.', 'TensorFlow is a machine learning library.', 'Yes, Python is great!', 'Nice talking, please type quit to exit', 'Nice talking, please type quit to exit', 'Nice talking, please type quit to exit']

# Tokenize the sentences (this is a very simplified approach)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

# Pad the sequences
max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1


In [8]:
# Define model parameters
embedding_dim = 256
units = 1024

# Define the encoder model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder model
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, None, 256)            12544     ['input_3[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, None, 256)            12544     ['input_4[0][0]']             
                                                                                            

In [9]:
# Prepare decoder input data that just contains the start token and the rest is all 0
# It's a simplified approach, in practice, you would use teacher forcing or another technique
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1  # Assuming 1 is the start token

# Train the model
model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=100)


Epoch 1/100
7/7 [==============================] - 4s 56ms/step - loss: 3.5074 - accuracy: 0.3297
Epoch 2/100
7/7 [==============================] - 0s 33ms/step - loss: 2.7681 - accuracy: 0.4066
Epoch 3/100
7/7 [==============================] - 0s 13ms/step - loss: 2.7687 - accuracy: 0.3956
Epoch 4/100
7/7 [==============================] - 0s 13ms/step - loss: 2.6281 - accuracy: 0.3956
Epoch 5/100
7/7 [==============================] - 0s 14ms/step - loss: 2.4787 - accuracy: 0.4066
Epoch 6/100
7/7 [==============================] - 0s 31ms/step - loss: 2.4663 - accuracy: 0.4066
Epoch 7/100
7/7 [==============================] - 0s 12ms/step - loss: 2.4097 - accuracy: 0.3846
Epoch 8/100
7/7 [==============================] - 0s 11ms/step - loss: 2.2714 - accuracy: 0.4286
Epoch 9/100
7/7 [==============================] - 0s 11ms/step - loss: 2.2039 - accuracy: 0.4176
Epoch 10/100
7/7 [==============================] - 0s 11ms/step - loss: 2.2271 - accuracy: 0.3846
Epoch 11/100
7/7 [=

In [10]:
# Step 5: Implement the chat function
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # end token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

# Step 6: Chat with the bot
input_text = "What is TensorFlow?"
print(f"You: {input_text}")
print(f"Bot: {chat_with_bot(input_text)}")

You: What is TensorFlow?
1/1 [==============================] - 0s 24ms/step
Bot: tensorflow a machine learning library


In [11]:
# Interactive chat with the bot
print("Start chatting with the bot! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")


Start chatting with the bot! Type 'quit' to exit.
You: Hello\
1/1 [==============================] - 0s 19ms/step
Bot: hello how
You: hello
1/1 [==============================] - 0s 22ms/step
Bot: hello how
You: goodbye
1/1 [==============================] - 0s 20ms/step
Bot: nice talking please type quit to
You: What is your favorite NBA team? 
1/1 [==============================] - 0s 26ms/step
Bot: 
You: NBA team? 
1/1 [==============================] - 0s 33ms/step
Bot: 
You: hello
1/1 [==============================] - 0s 22ms/step
Bot: hello how
You: Who is the best NBA team? 
1/1 [==============================] - 0s 23ms/step
Bot: the new york knicks
You: best NBA team? 
1/1 [==============================] - 0s 21ms/step
Bot: tensorflow a machine learning library
You: do you like python
1/1 [==============================] - 0s 20ms/step
Bot: yes python great
You: goodbye
1/1 [==============================] - 0s 24ms/step
Bot: nice talking please type quit to
You: quit
